In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%reload_ext autoreload

In [13]:
from skimage import io
from skimage import img_as_uint
from glob import glob
import numpy as np
import tifffile
from denoiser import Denoiser
from lsm_utils import normalize_16bit_images, compute_norm_range
import yaml

# Load data 
SHG samples

In [24]:
def sandbox_compute_norm_range(fname, percentiles=(0, 100), sample_r=0.1):
    max_val = []
    min_val = []
    fail_names = []
    try:
        img = img_as_uint(io.imread(fname))
    except Exception as e:
        print(e)
        print(fname)
        fail_names.append(fname)
    max_val.append(np.percentile(img, percentiles[1]))
    min_val.append(np.percentile(img, percentiles[0]))
    max_val: float | np.ndarray = np.percentile(np.array(max_val), 98)
    min_val: float | np.ndarray = np.percentile(np.array(min_val), 2)
    
    return min_val, max_val, fail_names

# vmin, vmax, fail_names = sandbox_compute_norm_range('sample_data/1B_C1.tif', percentiles=(1, 99.5), sample_r=0.05)
vmin, vmax, fail_names = compute_norm_range('sample_data', ext='tif', percentiles=(1, 99.5), sample_r=0.05)

sample_data\*.tif


0it [00:00, ?it/s]

max: []	min: []


IndexError: index -1 is out of bounds for axis 0 with size 0

In [22]:
config = yaml.load(open("model_config.yaml", "r"), Loader=yaml.FullLoader)
config['dataset'] = 'sample_data'
config['norm-range'] = [int(vmin), int(vmax)]
config['threads'] = 0